In [1]:
from collections import defaultdict
from typing import List
hypo_1 = 'John plays football or chess'
hypo_2 = 'When its raining, John does not play football'
hypo_3 = 'It is raining'

In [2]:
def detect_sentence_structure(sentence_tokens):
    if sentence_tokens is None or type(sentence_tokens) is not list or len(sentence_tokens) == 0:
            raise ValueError("Sentence structure detection only works for token lists. "
                             "And the token list is not empty")

    # TODO support multiple sentence base structures

    # Basic structure: A is B
    if len(sentence_tokens) == 3 and sentence_tokens[1] == "is":
        return 1

    # Basic structure: A does B
    if len(sentence_tokens) == 3:
        return 2

    # Inverted basic structure: A is not B
    if len(sentence_tokens) == 4 and sentence_tokens[1] == "is":
        return 3

    # Inverted basic structure: A does not B
    # Split because maybe we want to check: A does not do B or doesn't
    if len(sentence_tokens) == 4:
        return 4

    raise ValueError(f'Sentence structure is not detected: {str(sentence_tokens)}')

In [5]:
class Expression:
    def __init__(self, hypothesis):
        
        if hypothesis is None or type(hypothesis) is not str:
            raise ValueError("A hypothesis needs to be of type string and can't be empty.")
        
        self.init_hypo = hypothesis.lower()         # Only use lower case
        self.tokens: List = self.init_hypo.split(" ")     # Split hypo into tokens
        self.split_references()

        # A base expression has a few cases
        self.is_base_expression = False
        # Simple test if we have 3 or 4 (in not case) tokens then it is a base expression
        if len(self) == 3 or len(self) == 4:
            self.is_base_expression = True

    def split_references(self):
        """
        Splits the sentence that references previous subjects into multiple base tokens
        TODO Should probably be rewritten to check if separated by semicolon
        :return:
        """
        for reference in ['or', 'and']:
            if reference in self.tokens:
                reference_idx = self.tokens.index(reference)
                right_tokens = self.tokens[reference_idx + 1:]

                # If the right sentence is not just one word we dont support that atm

                if len(self) != 1:
                    continue

                base_tokens = self.tokens[:reference_idx - 1]
                left_tokens = self.tokens[:reference_idx]
                
                self.tokens = left_tokens + [reference] + base_tokens + right_tokens

    def reverse_expression(self):
        """
        Function that inserts a not or removes it
        :return: The hypothesis reversed
        """
        # Cant reverse expression if not base expression
        if not self.is_base_expression:
            return

        # TODO decide if we want to use a flag for a expression or use not (currently)
        sentence_structure = detect_sentence_structure(self.tokens)

        if sentence_structure == 1 or sentence_structure == 2:
            self.tokens.insert(2, "not")
            return
        elif sentence_structure == 3 or sentence_structure == 4:
            self.tokens.remove("not")
            return

        raise ValueError(f'Hypothesis cant be reversed: {str(self)}')

    def is_tautologie_of(self, clause):

        if not self.is_base_expression or not clause.is_base_expression:
            return False

        shorter_clause = None
        longer_clause = None

        if len(self) == len(clause) + 1:
            shorter_clause = clause
            longer_clause = self
        elif len(self) == len(clause) - 1:
            shorter_clause = self
            longer_clause = clause
        else:
            return False

        # Go over each token check for equals and also if not is on the correct location
        # Probably not to smart :sweat_smile:
        j = 0
        for token in longer_clause.tokens:
            if token == "not":
                continue
            if token != shorter_clause.tokens[j]:
                return False
            j += 1


        return True

    def __str__(self):
        return str(self.tokens)

    def __len__(self):
        return len(self.tokens)

True

In [7]:
exp = Expression(hypo_3)
print(exp)
exp.reverse_expression()
print(exp)
exp.reverse_expression()
print(exp)

['it', 'is', 'raining']
['it', 'is', 'not', 'raining']
['it', 'is', 'raining']


In [6]:
exp_1 = Expression(hypo_3)
exp_2 = Expression(hypo_3)
exp_2.reverse_expression()

print(exp_1.is_tautologie_of(exp_2))
print(exp_2.is_tautologie_of(exp_1))
print(exp_2.is_tautologie_of(exp_2))
print(exp_1.is_tautologie_of(exp_1))

True
True
False
False


In [ ]:
class TableauxSolver:
    
    def __init__(self, hypothesis, thesis):
        
        self.hypothesis: [Expression] = hypothesis
        self.thesis: Expression = thesis

    def proof(self):
        try:
            clauses = []
            for claus in self.hypothesis:
                clauses.append(claus)
            self.thesis.reverse_expression()
            clauses.append(self.thesis)
            result, applied_rules = self.recursive_proof(clauses)
        except RuntimeError as e:
            print(e)
            raise e
        return result, applied_rules

    @staticmethod
    def check_for_tautology(hypothesis: Expression, clauses: [Expression]):
        for clause in clauses:
            if hypothesis.is_tautologie_of(clause):
                return True
        return True

    @staticmethod
    def recursive_proof(clauses):
        # map indices to lists of indices, to store attempted unifications
        tried = defaultdict(list)

        i = 0
        while i < len(clauses):
            if not clauses[i].is_tautology():
                # since we try clauses in order, we should start after the last
                # index tried
                if tried[i]:
                    j = tried[i][-1] + 1
                else:
                    j = i + 1  # nothing tried yet for 'i', so start with the next

                while j < len(clauses):
                    # don't: 1) unify a clause with itself,
                    #       2) use tautologies
                    if i != j and j and not clauses[j].is_tautology():
                        tried[i].append(j)
                        newclauses = clauses[i].unify(clauses[j])
                        if newclauses:
                            for newclause in newclauses:
                                newclause._parents = (i + 1, j + 1)
                                clauses.append(newclause)
                                if not len(newclause):  # if there's an empty clause
                                    return True, clauses
                            i = -1  # since we added a new clause, restart from the top
                            break
                    j += 1
            i += 1
        return False, clauses
    
